In [1]:
# Full Optimized Code for Bird Classification
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress AVX warnings
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # Mitigate OpenMP conflict

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

In [3]:
# Enhanced Data Augmentation with Mixup and CutMix
class AugmentedGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, alpha=0.4, cutmix_prob=0.5):
        self.generator = generator
        self.alpha = alpha
        self.cutmix_prob = cutmix_prob

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        images, labels = self.generator[index]
        
        if np.random.rand() < self.cutmix_prob:
            return self.cutmix(images, labels)
        else:
            return self.mixup(images, labels)
    
    def mixup(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        mixed_images = lam * images + (1 - lam) * shuffled_images
        mixed_labels = lam * labels + (1 - lam) * shuffled_labels
        return mixed_images, mixed_labels

    def cutmix(self, images, labels):
        indices = np.random.permutation(images.shape[0])
        shuffled_images = images[indices]
        shuffled_labels = labels[indices]
        lam = np.random.beta(self.alpha, self.alpha)
        bbx1, bby1, bbx2, bby2 = self.rand_bbox(images.shape, lam)
        images[:, bbx1:bbx2, bby1:bby2, :] = shuffled_images[:, bbx1:bbx2, bby1:bby2, :]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.shape[1] * images.shape[2]))
        labels = lam * labels + (1 - lam) * shuffled_labels
        return images, labels

    def rand_bbox(self, size, lam):
        W = size[1]
        H = size[2]
        cut_rat = np.sqrt(1. - lam)
        cut_w = int(W * cut_rat)
        cut_h = int(H * cut_rat)
        cx = np.random.randint(W)
        cy = np.random.randint(H)
        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
        return bbx1, bby1, bbx2, bby2


In [4]:
# Dataset paths (UPDATE THESE TO YOUR LOCAL PATHS)
train_path = 'D:/Dataset/Odonata/Train'
test_path = 'D:/Dataset/Odonata/Test'

In [5]:
# Data Generators with Validation Split
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    rotation_range=60,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.4, 1.6],
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,  # Reduced batch size for memory stability
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(480, 480),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)


Found 105 images belonging to 15 classes.
Found 15 images belonging to 15 classes.
Found 30 images belonging to 15 classes.


In [6]:
# Build Enhanced Model Architecture
base_model = EfficientNetV2L(
    include_top=False,
    weights='imagenet',
    input_shape=(480, 480, 3)
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(2048, activation='swish', kernel_regularizer='l2'),
    Dropout(0.5),
    Dense(1024, activation='swish', kernel_regularizer='l2'),
    Dropout(0.4),
    Dense(15, activation='softmax')  # Ensure this matches your class count
])

In [7]:
# COMPILE THE MODEL BEFORE TRAINING
optimizer = Adam(learning_rate=1e-4, amsgrad=True)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

In [8]:
# First Training Phase (Frozen Backbone)
history = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    epochs=50
)

Epoch 1/50
27/27 [==============================] - 140s 4s/step - loss: 32.0842 - accuracy: 0.0381 - val_loss: 31.4727 - val_accuracy: 0.2000
Epoch 2/50
27/27 [==============================] - 102s 4s/step - loss: 31.3634 - accuracy: 0.1429 - val_loss: 31.0119 - val_accuracy: 0.0667
Epoch 3/50
27/27 [==============================] - 104s 4s/step - loss: 30.6970 - accuracy: 0.2190 - val_loss: 30.5622 - val_accuracy: 0.0667
Epoch 4/50
27/27 [==============================] - 110s 4s/step - loss: 30.1822 - accuracy: 0.2762 - val_loss: 30.1106 - val_accuracy: 0.2667
Epoch 5/50
27/27 [==============================] - 108s 4s/step - loss: 29.7847 - accuracy: 0.2857 - val_loss: 29.6454 - val_accuracy: 0.4000
Epoch 6/50
27/27 [==============================] - 108s 4s/step - loss: 29.3224 - accuracy: 0.2762 - val_loss: 29.2348 - val_accuracy: 0.3333
Epoch 7/50
27/27 [==============================] - 106s 4s/step - loss: 28.8865 - accuracy: 0.3048 - val_loss: 28.8375 - val_accuracy: 0.0667

In [9]:
model.save('D:/Model_Main/Xception_net_client_odonata_cnn.h5')

In [10]:
# Fine-tuning Phase
base_model.trainable = True
for layer in base_model.layers[:300]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history_fine = model.fit(
    AugmentedGenerator(train_generator),
    validation_data=val_generator,
    initial_epoch=history.epoch[-1],
    epochs=history.epoch[-1] + 30,
    
)

Epoch 50/79
27/27 [==============================] - 302s 10s/step - loss: 14.4507 - accuracy: 0.4571 - val_loss: 13.6379 - val_accuracy: 0.4667
Epoch 51/79
27/27 [==============================] - 255s 9s/step - loss: 14.2477 - accuracy: 0.5810 - val_loss: 13.8203 - val_accuracy: 0.5333
Epoch 52/79
27/27 [==============================] - 252s 9s/step - loss: 14.1826 - accuracy: 0.5143 - val_loss: 13.6508 - val_accuracy: 0.6667
Epoch 53/79
27/27 [==============================] - 256s 9s/step - loss: 14.1332 - accuracy: 0.6000 - val_loss: 13.6733 - val_accuracy: 0.6667
Epoch 54/79
27/27 [==============================] - 254s 10s/step - loss: 13.9669 - accuracy: 0.6476 - val_loss: 13.7287 - val_accuracy: 0.5333
Epoch 55/79
27/27 [==============================] - 258s 10s/step - loss: 13.7532 - accuracy: 0.7524 - val_loss: 13.4641 - val_accuracy: 0.7333
Epoch 56/79
27/27 [==============================] - 257s 9s/step - loss: 13.8624 - accuracy: 0.7143 - val_loss: 13.2887 - val_accura

In [11]:
model.save('D:/Model_Main/Xception_net_client_odonata_cnn_tunned.h5')

In [12]:
# Evaluate the model
eval_results = model.evaluate(test_generator)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

8/8 [==============================] - 24s 3s/step - loss: 12.3168 - accuracy: 0.9333
Test Loss: 12.3168 | Test Accuracy: 0.9333


In [86]:
# Define class labels manually
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import numpy as np
from tensorflow.keras.models import load_model

class_labels = {
    0: 'Brown Marsh Hawk',
    1: 'Common Picturewing',
    2: 'Coral Cloudwing',
    3: 'Green Marsh Hawk',
    4: 'Lathrecista',
    5: 'Libellago balus',
    6: 'Macrodiplax cora',
    7: 'Pantala',
    8: 'Rambur',
    9: 'Yellow Flutterer',
    10: 'fluctuans',
    11: 'fulvia Drury',
    12: 'servilia Drury',
    13: 'verticalis',
    14: 'winged Forest Glory'
    
}

# Load the trained model
model = load_model('D:/Model_Main/Xception_net_client_odonata_cnn_tunned.h5')



In [91]:
# Load and preprocess the image
image_path = 'D:/Less_data_test/odonata/b45.jpg'
test_image = load_img(image_path, target_size=(480, 480))  # Match model input size
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)  # Use EfficientNetV2 preprocessing

# Predict the class
result = model.predict(test_image)
predicted_class = np.argmax(result)

# Get the predicted label
prediction = class_labels[predicted_class]

print(f'The predicted class is: {prediction}')

1/1 [==============================] - 1s 1s/step
The predicted class is: Yellow Flutterer
